In [1]:
import pandas as pd
import pymc3 as pm
import numpy as np
import theano
import theano.tensor as tt

In [2]:
data = pd.read_csv("HerrmannThoeniGaechterDATA.csv", skiprows=3)

In [3]:
len(data)

65880

In [4]:
data.head()

,sessionid,groupid,mgroupid,subjectid,p,secseq,city,period,senderscontribution,otherscontribution,...,civic,ruleoflaw,female,age,numknown,singlechild,ageu21,urbanbackground,middleclass,membership
0,011108KA,703,703,701,N-experiment,0,Samara,1,20,18,...,8.050114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
1,011108KA,703,703,701,N-experiment,0,Samara,1,20,10,...,8.050114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
2,011108KA,703,703,701,N-experiment,0,Samara,1,20,10,...,8.050114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
3,011108KA,702,702,702,N-experiment,0,Samara,1,10,10,...,8.050114,-0.884283,1,21.0,10.0,0.0,0,1.0,1.0,1.0
4,011108KA,702,702,702,N-experiment,0,Samara,1,10,20,...,8.050114,-0.884283,1,21.0,10.0,0.0,0,1.0,1.0,1.0


In [5]:
## constants
groupSize = 4
ntrials = 10
pi = 1.4
ntokens = 20
vals = list(range(21)) #possible values to contribute - from 0 to 20 tokens

## Prelim checks

In [6]:
len(set(data["subjectid"])) #should be 1120

1120

In [7]:
len(set(data["groupid"]))

280

In [9]:
def get_gini(city):
    if city == "Melbourne":
        return 34.3
    if city == "Minsk":
        return 25.3
    if city == "Chengdu":
        return 38.5
    if city == "Copenhagen":
        return 28.7
    if city == "Bonn":
        return 31.9
    if city == "Athens":
        return 34.4
    if city == "Seoul":
        return 31.6
    if city == "Samara":
        return 37.5
    if city == "Zurich":
        return 32.7
    if city == "St. Gallen":
        return 32.7
    if city == "Istanbul":
        return 41.9
    if city == "Nottingham":
        return 34.8
    if city == "Dnipropetrovs'k":
        return 26.1
    if city == "Boston":
        return 41.1
    
data["gini"] = data["city"].apply(lambda x: get_gini(x))

In [10]:
data = data[data["gini"].isna() == False]
#data = data.dropna()

In [11]:
len(data)

59880

In [34]:
redDat = data.iloc[::3, :] #get only every third row, the others are other responses that we don't need

In [35]:
redDat.reset_index(inplace = True, drop = True)

In [36]:
group_names = set(redDat["groupid"])
group_names = sorted(list(group_names))
ngroups = len(group_names)

### DOES SOMETHING FUCKED HERE

subject_names = set(redDat["subjectid"])
nsubjects = len(subject_names)
#ngroups = 269

## redDat
punish_dat = redDat[redDat["p"] == "P-experiment"]
non_punish_dat = redDat[redDat["p"] == "N-experiment"]

In [37]:
non_punish_dat = non_punish_dat.sort_values(["groupid", "period"]).reset_index(drop = True)

In [38]:
### get ids
participant = [[1,2,3,4] for i in set(non_punish_dat["period"]) for j in set(non_punish_dat["groupid"])]
participant = [item for sublist in participant for item in sublist]

In [39]:
non_punish_dat["participant"] = participant

In [40]:
coder = pd.Categorical(non_punish_dat["groupid"]).codes
y = non_punish_dat["senderscontribution"].values


In [41]:
tester = non_punish_dat.groupby(["groupid", "participant", "period"])[["gini", "groupid", "participant", "period", "senderscontribution"]].mean().reset_index(drop = True)


In [44]:
## get variables
ngroups = len(set(tester["groupid"]))

Gini = tester.groupby(["groupid"]).mean()["gini"].values
g = coder
t = tester["period"].values
s = tester["participant"].values
#Gini = non_punish_dat.groupby(["groupid"]).mean()["gini"].values
#g = np.arange(ngroups)
#t = np.arange(10)
#s = np.arange(4)

## Decay model

$$c_{g,s,t}^\mu = c^0_{g,s} \cdot e^{-\gamma_{s,g} \cdot t}$$

$$c^0_{g,s} = \beta^c_0 + \beta^c_{Gini} \cdot Gini_g$$

$$\gamma_{s,g} = \beta^\gamma_0 + \beta^\gamma_{Gini} \cdot Gini_g$$

In [47]:
print(ngroups, groupSize)

244 4


In [50]:
len(Gini[g]) #this might be an issue

9760

In [101]:
#is t a matrix?!!
t_question_mark = np.array([[0,1,2,3,4,5,6,7,8,9],] * 244).T

In [92]:
t_question_mark = np.array([[0,1,2,3,4,5,6,7,8,9],] * 4).T

In [71]:
y_question_mark = y.reshape((244, 4, 10))

In [109]:
Gini_question_mark = np.stack([Gini, Gini, Gini, Gini])

In [110]:
Gini_question_mark.shape #perhaps needs to be flipped, who knows!

(4, 244)

In [ ]:
### COMPLETELY NEW APPROCH: 
# The trick is to get the right shapes all the time....
# follow the comments

In [99]:
t_question_mark.shape

(244, 10)

In [112]:
with pm.Model() as m:
    ## priors
    beta0_c0 = pm.HalfNormal("beta0_c0", 1)
    betaGini_c0 = pm.Normal("betaGini_c0", 0, 1)
    
    beta0_gamma = pm.HalfNormal("beta0_gamma", 1)
    betaGini_gamma = pm.Normal("betaGini_gamma", 0, 1)
    
    ## group_level
    sigma_c = pm.Gamma("sigma_c", 1, 1, shape = (ngroups, groupSize))
    
    ##shape: correct by construction!
    
    c_0 = pm.Deterministic("c_0", beta0_c0 + betaGini_c0 * Gini_question_mark)
    
    ## shape: not correct! needs to be (4, 244).
    
    #tester = pm.math.dot(sigma_c, c_0)
    
    gamma = pm.Deterministic("gamma", beta0_gamma + betaGini_gamma * Gini_question_mark)
    
    ## shape: not correct! needs to be (4, 244). 
    
    #### NOTE: Both shape issues above comes from having Gini in normal format. Needs to be stacked??
    
    ### with time 
    
    #pm.math.dot(c_0, -gamma)
    
    mu_c = pm.Deterministic("mu_c", 
                            pm.math.dot(
                            c_0,
                                pm.math.exp(pm.math.dot(t_question_mark, -gamma)))) ## not t right now :()
    
    ## likelihood
    
    #test = pm.math.dot(sigma_c, mu_c)
    
    y_pred = pm.Normal("y_pred", mu = mu_c, sigma = sigma_c, observed = y_question_mark)
    
    

ValueError: shapes (10,244) and (4,244) not aligned: 244 (dim 1) != 4 (dim 0)

### Preferences:

$$pvec = p_0 + \beta_p \cdot tokenvalues $$

$$p_t = pvec[Gb_t]$$

where Gb_t is the belief of what the other person will do at time T

$$Gb_t = \gamma ( Gb_{t-1}) + (1-\gamma) (G_{a_{t-1}})$$

$$c_t = \omega_t (Gb_t) + (1-\omega_t) (p_t)$$

$$\omega_t = \omega_{t-1}(1-\lambda)$$

### Getting the hierachical model

$$B^P_{g,s} \sim Beta(Shape_{g,s,1}, Shape_{g,s,2})$$

$$Shape_1 = \mu B^P_{g,s} + \sigma_{g,s}$$
$$Shape_2 = (1 - \mu B^P_{g,s}) + \sigma_{g,s}$$

$$Probit(\mu B^P_{g,s}) = \beta_0^P + \beta^P_{Gini} \cdot Gini$$

In [40]:
t

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])